In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import xgboost as xgb
import time

from math import sqrt
from numpy import loadtxt
from itertools import product
from sklearn import preprocessing
from xgboost import plot_tree
from matplotlib import pyplot

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import KFold
from sklearn.feature_extraction.text import  TfidfVectorizer


kernel_with_output = False


In [2]:
sales_train = pd.read_csv('../input/sales_train.csv')
items = pd.read_csv('../input/items.csv')
shops = pd.read_csv("../input/shops.csv")
item_categories = pd.read_csv('../input/item_categories.csv')
test =pd.read_csv('../input/test.csv')
sample_submission = pd.read_csv('../input/sample_submission.csv')


In [3]:
grid = []


In [6]:
sales_train['date_block_num'].unique()


array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33])

In [14]:
cur_shops = sales_train[sales_train['date_block_num']==0]['shop_id'].unique()
cur_items = sales_train[sales_train['date_block_num'] == 0]['item_id'].unique()



In [15]:
grid.append(np.array(list(product(*[cur_shops, cur_items, [0]])),dtype='int32'))

In [12]:
sales_train[sales_train['date_block_num']==1]['shop_id'].unique()

array([50, 47, 51, 52, 42, 46, 43, 54, 56, 59,  7, 53, 16, 18, 19, 14, 15,
       13, 23, 24, 21, 25, 22,  4,  3,  5,  1,  2,  6,  0,  8, 12, 10, 31,
       32, 35, 44, 41, 45, 37, 38, 26, 27, 29, 30, 28])

In [13]:
sales_train[sales_train['date_block_num'] == 1]['item_id'].unique()

array([ 3880,  4128,  4124, ..., 12885, 12791, 13433])

In [16]:
grid

[array([[   59, 22154,     0],
        [   59,  2552,     0],
        [   59,  2554,     0],
        ...,
        [   45,   628,     0],
        [   45,   631,     0],
        [   45,   621,     0]], dtype=int32)]

In [20]:

cur_shops = sales_train[sales_train['date_block_num']==2]['shop_id'].unique()
cur_items = sales_train[sales_train['date_block_num'] == 2]['item_id'].unique()

In [21]:
grid.append(np.array(list(product(*[cur_shops, cur_items, [2]])),dtype='int32'))

In [22]:
grid

[array([[   59, 22154,     0],
        [   59,  2552,     0],
        [   59,  2554,     0],
        ...,
        [   45,   628,     0],
        [   45,   631,     0],
        [   45,   621,     0]], dtype=int32),
 array([[   50,  3880,     1],
        [   50,  4128,     1],
        [   50,  4124,     1],
        ...,
        [   28, 12885,     1],
        [   28, 12791,     1],
        [   28, 13433,     1]], dtype=int32),
 array([[    5, 20175,     2],
        [    5, 20083,     2],
        [    5,    31,     2],
        ...,
        [    4, 12388,     2],
        [    4, 12340,     2],
        [    4, 10649,     2]], dtype=int32)]

In [23]:
sales_train

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day
0,02.01.2013,0,59,22154,999.00,1.0
1,03.01.2013,0,25,2552,899.00,1.0
2,05.01.2013,0,25,2552,899.00,-1.0
3,06.01.2013,0,25,2554,1709.05,1.0
4,15.01.2013,0,25,2555,1099.00,1.0
...,...,...,...,...,...,...
2935844,10.10.2015,33,25,7409,299.00,1.0
2935845,09.10.2015,33,25,7460,299.00,1.0
2935846,14.10.2015,33,25,7459,349.00,1.0
2935847,22.10.2015,33,25,7440,299.00,1.0


In [24]:
group = sales_train.groupby(['shop_id','item_id','date_block_num'])

In [35]:
group.agg({'item_cnt_day':'sum','item_price':'mean'}).reset_index()

,shop_id,item_id,date_block_num,item_cnt_day,item_price
0,0,30,1,31.0,265.0
1,0,31,1,11.0,434.0
2,0,32,0,6.0,221.0
3,0,32,1,10.0,221.0
4,0,33,0,3.0,347.0
...,...,...,...,...,...
1609119,59,22164,27,2.0,699.0
1609120,59,22164,30,1.0,699.0
1609121,59,22167,9,1.0,299.0
1609122,59,22167,11,2.0,299.0


In [37]:
trainset = group.agg({'item_cnt_day':'sum','item_price':'mean'}).reset_index()

In [39]:
trainset.rename(columns = {'item_cnt_day':'item_cnt_month'})

,shop_id,item_id,date_block_num,item_cnt_month,item_price
0,0,30,1,31.0,265.0
1,0,31,1,11.0,434.0
2,0,32,0,6.0,221.0
3,0,32,1,10.0,221.0
4,0,33,0,3.0,347.0
...,...,...,...,...,...
1609119,59,22164,27,2.0,699.0
1609120,59,22164,30,1.0,699.0
1609121,59,22167,9,1.0,299.0
1609122,59,22167,11,2.0,299.0


In [25]:
np.vstack(grid)

array([[   59, 22154,     0],
       [   59,  2552,     0],
       [   59,  2554,     0],
       ...,
       [    4, 12388,     2],
       [    4, 12340,     2],
       [    4, 10649,     2]], dtype=int32)

In [30]:
group

In [27]:
sales_train[sales_train['date_block_num']==2]['shop_id'].unique()

array([ 5,  2, 41, 38, 44, 37, 35, 46, 42, 45, 31, 30, 32, 54, 53, 56, 57,
       47, 43, 50, 51, 52, 18, 19, 16, 15, 21, 22, 23,  7,  6,  8, 10, 14,
       12, 13, 28, 27, 29, 25, 24, 26, 58, 59,  3,  4])

In [4]:
    grid = []
    for block_num in sales_train['date_block_num'].unique():
        cur_shops = sales_train[sales_train['date_block_num']==block_num]['shop_id'].unique()
        cur_items = sales_train[sales_train['date_block_num'] == block_num]['item_id'].unique()
        grid.append(np.array(list(product(*[cur_shops, cur_items, [block_num]])),dtype='int32'))
    index_cols = ['shop_id', 'item_id', 'date_block_num']
    grid = pd.DataFrame(np.vstack(grid), columns=index_cols, dtype=np.int32)
    sales_train['item_cnt_day'] = sales_train['item_cnt_day'].clip(0,20)
    groups = sales_train.groupby(['shop_id', 'item_id','date_block_num'])
    trainset = groups.agg({'item_cnt_day':'sum', 'item_price':'mean'}).reset_index()

    trainset = trainset.rename(columns={'item_cnt_day':'item_cnt_month'})
    trainset['item_cnt_month'] = trainset['item_cnt_month'].clip(0,20)
    trainset = pd.merge(trainset, items[['item_id','item_category_id']], on='item_id')
    trainset.to_csv('trainset_with_grid.csv')


In [5]:
prev_month_selector = (trainset.month == 10) & (trainset.year == 2015)
    


AttributeError: 'DataFrame' object has no attribute 'month'

In [ ]:
    prev_month_selector = (trainset.month == 10) & (trainset.year == 2015)
    train_subset = trainset[prev_month_selector]
    groups = train_subset[['shop_id','item_id','item_cnt_month']].groupby(by=['shop_id','item_id'])
    train_subset = groups.agg({'item_cnt_month':'sum'}).reset_index()
    train_subset.head()


In [6]:
trainset

,shop_id,item_id,date_block_num,item_cnt_month,item_price,item_category_id
0,0,30,1,20.0,265.0,40
1,1,30,1,10.0,265.0,40
2,2,30,2,1.0,359.0,40
3,2,30,5,1.0,399.0,40
4,2,30,15,1.0,169.0,40
...,...,...,...,...,...,...
1609119,59,9760,12,1.0,179.0,37
1609120,59,12846,10,1.0,149.0,37
1609121,59,16270,6,1.0,1000.0,75
1609122,59,17732,9,1.0,198.0,28


In [ ]:
    prev_month_selector = (trainset.month == 10) & (trainset.year == 2015)
    train_subset = trainset[prev_month_selector]
    groups = train_subset[['shop_id','item_id','item_cnt_month']].groupby(by=['shop_id','item_id'])
    train_subset = groups.agg({'item_cnt_month':'sum'}).reset_index()
    train_subset.head()


In [ ]:
    prev_month_selector = (trainset.month == 10) & (trainset.year == 2015)
    train_subset = trainset[prev_month_selector]
    groups = train_subset[['shop_id','item_id','item_cnt_month']].groupby(by=['shop_id','item_id'])
    train_subset = groups.agg({'item_cnt_month':'sum'}).reset_index()
    train_subset.head()


In [ ]:
    prev_month_selector = (trainset.month == 10) & (trainset.year == 2015)
    train_subset = trainset[prev_month_selector]
    groups = train_subset[['shop_id','item_id','item_cnt_month']].groupby(by=['shop_id','item_id'])
    train_subset = groups.agg({'item_cnt_month':'sum'}).reset_index()
    train_subset.head()
